In [3]:
#import gas_property_cantera
import pandas as pd
import numpy as np
import math
import cantera as ct
#pd.set_option("display.float_format", "{:.6f}".format)
#np.set_printoptions(suppress=True)

In [12]:
def get_property(fuel='H2',oxy_agent='air',ratio=1,temp=298,pressure=101325):
    oxy_comp = {'air':'O2:1, N2:3.76','O2':'O2:1'}

    gas = ct.Solution('Li-Dryer.yaml')
    gas.TP = temp,pressure
    gas.set_equivalence_ratio(ratio,fuel,oxy_comp[oxy_agent])

    return gas.density, gas.viscosity


def get_tau(velocity,density,re):

    return

def resolution_check_yplus(gridsize,fuel='H2',oxy='air',temp=298,ratio=1,pressure=101325,velocity=10,channel_height=1):
    density,viscosity = get_property(fuel,oxy,0.4,300,101325)
    re= velocity*channel_height*density/viscosity
    np.set_printoptions(suppress=True, precision=6)
    if re<0:
        return -1
    if re<1400:
        tau =0.5* 16/(3*re)*(velocity**2) *density
    else:
        tau = 0.5*0.073*(re**(-0.25))*(velocity**2) *density
    viscous_len = (viscosity/density/(math.sqrt(tau/density)))
    #print(viscous_len)
    y_plus_1 = gridsize/viscous_len
    #y_plus_10 = np.floor(10*gridsize/viscous_len
    #y_plus_1 = gridsize*(math.sqrt(tau/density))/viscosity
    #y_plus_10 = 10*gridsize*(math.sqrt(tau/density))/viscosity
    print(f'With given condition, viscous lengthscale is {viscous_len} m.')
    #print(f'The first grid point is located at y+ = {y_plus_1}.')
    #print(f'The 10th grid point is located at y+ = {y_plus_1}.')
    pts_1yp = np.floor(1*viscous_len/gridsize)
    pts_10yp = np.floor(10*viscous_len/gridsize)
    print(f'There will be {pts_1yp} points with in y+ = 1.')
    print(f'There will be {pts_10yp} points with in y+ = 10.')


    return viscous_len, pts_1yp,pts_10yp
    #return viscous_len, y_plus_1,y_plus_10

def resolution_check_gridsize(fuel='H2',oxy='air',temp=298,ratio=1,pressure=101325,velocity=10,channel_height=1):
    density,viscosity = get_property(fuel,oxy,ratio,300,101325)
    #print(f'density is {density}')
    #print(f'viscosity is {viscosity}')
    re= velocity*channel_height*density/viscosity
    if re<0:
        return -1
    if re<1400:
        tau =0.5* 16/(3*re)*(velocity**2) *density
    else:
        tau = 0.5*0.073*(re**(-0.25))*(velocity**2) *density
    viscous_len = (viscosity/density/(math.sqrt(tau/density)))
    grid_yp1 = 1*viscous_len/1
    grid_yp10 = 10* viscous_len/13
    print(f'The gridsize needs to have 1st grid point within y+ =1 is {grid_yp1} meter.')
    print(f'The gridsize needs to have 13 grid points within y+ =10 is {grid_yp10} meter.')


    return viscous_len, grid_yp1,grid_yp10


In [6]:
#vl1,yp1,yp10 = resolution_check(0.000018,'H2','air',300,0.4,101325,12.5,0.02)
#vg1,yp1,yp10 =resolution_check_gridsize('H2','air',300,0.4,101325,12.5,0.02)

In [8]:
phi_list = np.linspace(0.2,1,11)
vel_list = np.linspace(5,20,21)
df_f = pd.DataFrame('0', index=phi_list, columns=vel_list)


In [34]:
max_cells = 0
for i in range(df_f.shape[0]):
    for j in range(df_f.shape[1]):
        _,typ1,typ10 = resolution_check_gridsize('H2','air',298,phi_list[i],101325,vel_list[j],0.02)
        temp_cells = int(math.ceil(0.035/(typ1*2*2*2*2)))
        amr4 = int(math.ceil(0.035/(typ1)))
        print(f'The phi is {phi_list[i]} with velocity {vel_list[j]} m/s.')
        print(f'The cells need to reach appropriate resolution is \033[1m {temp_cells}\033[0m in 0.035m of domain at AMR0.')
        print(f'At AMR4, it need \033[1m {amr4}\033[0m cells.')
        max_cells =  max(max_cells,temp_cells)
        #print(f'The phi is {phi_list[i]} and the velocity is {vel_list[j]}.')
        #print(vel_list[j])
        df_f.iloc[i,j] = str(typ1)+'  ,  '+str(typ10)

print(max_cells)

The gridsize needs to have 1st grid point within y+ =1 is 5.293419704716479e-05 meter.
The gridsize needs to have 13 grid points within y+ =10 is 4.0718613113203686e-05 meter.
The phi is 0.2 with velocity 5.0 m/s.
The cells need to reach appropriate resolution is  42 in 0.035m of domain at AMR0.
At AMR4, it need  662 cells.
The gridsize needs to have 1st grid point within y+ =1 is 4.684095267514333e-05 meter.
The gridsize needs to have 13 grid points within y+ =10 is 3.603150205780256e-05 meter.
The phi is 0.2 with velocity 5.75 m/s.
The cells need to reach appropriate resolution is  47 in 0.035m of domain at AMR0.
At AMR4, it need  748 cells.
The gridsize needs to have 1st grid point within y+ =1 is 4.207614048095994e-05 meter.
The gridsize needs to have 13 grid points within y+ =10 is 3.236626190843073e-05 meter.
The phi is 0.2 with velocity 6.5 m/s.
The cells need to reach appropriate resolution is  52 in 0.035m of domain at AMR0.
At AMR4, it need  832 cells.
The gridsize needs to h

In [14]:
df_f

,5.0,5.5,6.0,6.5,7.0,7.5,8.0,8.5,9.0,9.5,...,10.5,11.0,11.5,12.0,12.5,13.0,13.5,14.0,14.5,15.0
0.1,"5.13121294860761e-05 , 3.947086883544316e-05","4.7206460542420025e-05 , 3.631266195570771e-05","4.374580857690447e-05 , 3.365062198223421e-05","4.0786797364842324e-05 , 3.137445951141717e-05","3.8225925346468776e-05 , 2.9404557958822135e-05","3.598654781643515e-05 , 2.7681959858796268e-05","3.401065952298358e-05 , 2.616204578691044e-05","3.225352845689024e-05 , 2.4810406505300183e-05","3.068008768859569e-05 , 2.3600067452765915e-05","2.9262446836947785e-05 , 2.2509574489959836e-05",...,"2.680884820189469e-05 , 2.0622190924534375e-05","2.5739505126633208e-05 , 1.9799619328179393e-05","2.47575794833593e-05 , 1.9044291910276386e-05","2.3852571262404237e-05 , 1.834813174031095e-05","2.301561217936896e-05 , 1.7704317061053046e-05","2.2239158958505753e-05 , 1.7107045352696734e-05","2.1516753517658414e-05 , 1.6551348859737242e-05","2.084283358930453e-05 , 1.603294891484964e-05","2.021258179370061e-05 , 1.554813984130816e-05","1.9621804332875285e-05 , 1.5093695640673296e-05"
0.2,"5.293346256663251e-05 , 4.071804812817885e-05","4.869806490302594e-05 , 3.7460049925404574e-05","4.512806511725464e-05 , 3.471389624404203e-05","4.20755566597672e-05 , 3.2365812815205536e-05","3.9433767584158927e-05 , 3.0333667372429945e-05","3.712363140688804e-05 , 2.8556639543760034e-05","3.508531005743698e-05 , 2.6988700044182294e-05","3.327265810860375e-05 , 2.559435239123365e-05","3.164950060484173e-05 , 2.4345769696032098e-05","3.0187065899729827e-05 , 2.32208199228691e-05",...,"2.7655939774134768e-05 , 2.1273799826257513e-05","2.655280817129219e-05 , 2.042523705484015e-05","2.553985616945488e-05 , 1.9646043207272983e-05","2.460625198529373e-05 , 1.892788614253364e-05","2.374284711912709e-05 , 1.8263728553174686e-05","2.2941859946835654e-05 , 1.7647584574488963e-05","2.2196628327255323e-05 , 1.7074329482504095e-05","2.1501414239324947e-05 , 1.6539549414865344e-05","2.0851248086325853e-05 , 1.603942160486604e-05","2.0241803556913112e-05 , 1.5570618120702394e-05"
0.3,"5.452636741810166e-05 , 4.194335955238589e-05","5.0163515679905374e-05 , 3.858731975377336e-05","4.6486085363374045e-05 , 3.575852720259542e-05","4.3341719028179076e-05 , 3.333978386783006e-05","4.062043168378104e-05 , 3.12464859106008e-05","3.824077753156761e-05 , 2.9415982716590465e-05","3.614111781865116e-05 , 2.7800859860500894e-05","3.427391848252607e-05 , 2.6364552678866204e-05","3.260191596963232e-05 , 2.507839689971717e-05","3.109547282026274e-05 , 2.3919594477125183e-05",...,"2.8488178560378986e-05 , 2.1913983507983837e-05","2.735185087330585e-05 , 2.1039885287158346e-05","2.6308416524767703e-05 , 2.0237243480590544e-05","2.5346717775048325e-05 , 1.9497475211575633e-05","2.445733082243934e-05 , 1.8813331401876418e-05","2.3632239873617107e-05 , 1.8178646056628545e-05","2.2864582306351917e-05 , 1.758814023565532e-05","2.2148447427682027e-05 , 1.7037267252063097e-05","2.1478716092864923e-05 , 1.6522089302203787e-05","2.085093180065316e-05 , 1.6039178308194737e-05"
0.4,"5.609150649106011e-05 , 4.314731268543085e-05","5.160342231856986e-05 , 3.969494024505374e-05","4.782043408301651e-05 , 3.678494929462808e-05","4.458581103636353e-05 , 3.429677772027964e-05","4.178641115021473e-05 , 3.2143393192472864e-05","3.933845078451015e-05 , 3.02603467573155e-05","3.717852189152155e-05 , 2.8598862993478118e-05","3.525772597861439e-05 , 2.712132767585722e-05","3.3537729869467504e-05 , 2.5798253745744234e-05","3.1988045382999665e-05 , 2.4606188756153585e-05",...,"2.9305910668596852e-05 , 2.254300820661296e-05","2.8136965535195683e-05 , 2.164381964245822e-05","2.7063580174948992e-05 , 2.081813859611461e-05","2.6074276573469372e-05 , 2.0057135825745668e-05","2.515936042578612e-05 , 1.9353354173681633e-05","2.4310585851153206e-05 , 1.8700450654733237e-05","2.35208932408424e-05 , 1.8092994800647998e-05","2.2784202239819446e-05 , 1.752630941524573e-05","2.209524676207587e-05 , 1.6996343663135284e-05","2.144

In [18]:
density,viscosity = get_property('H2','air',0.38,300,101325)
print(viscosity/density)
#print(viscosity)

1.8153969447993528e-05
